# Read data timeseries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import libarchive
import os
import xarray as xr
import pandas as pd
import datetime
import tqdm
import pickle

In [2]:
import io
import dask

In [3]:
from io import BytesIO

In [4]:
# 1) read all station names found in timeseries data, makes it faster to read this data later
# dir1 = './202112/2021-12/'
# files = sorted(os.listdir(dir1)) 
# start = '2021-12-01'
# end = '2022-01-01'

# create the csv file with header and columnnames
# file_counter = 0
# station_names = None
# ds_cml = None # dataset to store cml data in
# files_list = [] # raw data to store
# for file in tqdm.tqdm(files):
#     file_out = read_file(file) 
#     if file_out is not None:
#         files_list.append( file_out )
#         file_counter += 1
        
#     if file_counter > 500: # read data in cunchs
#         df = pd.concat(files_list)['Link'] # only need link data
#         if station_names is None:
#             station_names = set(list(df))
#         else:
#             station_names.update(list(df))
#         files_list = [] # reset container
#         file_counter = 0 # reset counter
        
# with open('./2022-06_station_names.pickle', 'wb') as handle:
#     pickle.dump(station_names, handle)

In [5]:
# input files
period = 'summer'
if period == 'winter':
    dir1 = '/home/erlend/offline_data/cml_telia/202112/2021-12/'
    files = sorted(os.listdir(dir1)) 
    files = [ ff for ff in files if ff.split('.')[-2] == 'cpio'] # only use cpio files
    start = '2021-12-01'
    end = '2022-01-01'

    with open('/home/erlend/offline_data/cml_telia/2021-12_station_names.pickle', 'rb') as handle:
        station_names = list(pickle.load(handle))

if period == 'summer':
    dir1 = '/home/erlend/offline_data/cml_telia/202206/2022-06/'
    files = sorted(os.listdir(dir1)) 
    files = [ ff for ff in files if ff.split('.')[-2] == 'cpio'] # only use cpio files
    start = '2022-05-31'
    end = '2022-07-01'

    with open('/home/erlend/offline_data/cml_telia/2021-12_station_names.pickle', 'rb') as handle:
        station_names = list(pickle.load(handle))

# input files
# dir1 = './202206/2022-06/'
# files = sorted(os.listdir(dir1)) 
# start = '2022-05-31'
# end = '2022-07-01'
#with open('./2022-06_station_names.pickle', 'rb') as handle:
#    station_names = pickle.load(handle)

In [6]:
# # function for reading files
# def read_file(file): 
#     with libarchive.Archive(dir1 + file) as a:
#         for entry in a: # move to first file
#             if entry.pathname[:7] == 'mw_data':
#                 data = BytesIO(a.read(size = entry.size))
#                 df = pd.read_csv(data, sep=",",index_col=False)[['Timestamp', 'Link', 'TX', 'RX']]
#                 df = df.rename(columns = {'Link': 'cml_id', 'TX': 'ts', 'RX': 'rs'})
#                 df['time'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d%H%M%S')
#                 df = df.drop(columns = ['Timestamp'])
                
#                 # keep stations that are in metadata (remove the rest)
#                 df = df[df['cml_id'].isin(station_names)]
                
#                 # create multi index for xarray transform
#                 df = df.reset_index()
#                 df = df.set_index(['cml_id', 'time']).sort_values([('time')], ascending=True)
#                 return df.to_xarray().drop('index')
#     return None

In [7]:

# for file in files:
#     with xr.open_dataset("concatenated.nc", mode="w") as ds:
#         dataset = read_file(file)
#         if dataset is not None:
#             ds = xr.concat([ds, dataset], dim="time")
#             ds.to_netcdf("concatenated.nc", mode="a")

In [8]:
# # Concatenate the datasets using dask.array.concatenate
# concatenated = xr.concat(datasets, dim="time", data_vars="minimal", coords="minimal", compat="override")

# # Save the Dask-backed xarray Dataset to a new zarr file
# concatenated.to_netcdf("concatenated.nc")

In [9]:
# # Open the datasets lazily as a Dask-backed xarray Dataset using xr.open_mfdataset
# ds = xr.open_mfdataset(datasets, concat_dim="new_dimension_name", combine="by_coords")

# # Save the dataset to zarr format
# ds.to_zarr("dataset.zarr", consolidated=True)

### Get metadata

In [10]:
# function for calculating lengths
def haversine(lat1, lon1, lat2, lon2):
    # return: distance [km] between two points, assuming earth is a sphere
    # https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
    R = 6373.0 # approximate radius of earth in km
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    
    return abs(distance)

# get metadata
df = pd.read_csv('/home/erlend/offline_data/cml_telia/telia_metadata.csv').rename(columns={'sublink_id':'cml_id'}).set_index('cml_id')
cmls = df.to_xarray()
lengths = haversine(cmls.site_a_latitude.values, 
          cmls.site_a_longitude.values, 
          cmls.site_b_latitude.values, 
          cmls.site_b_longitude.values)

cmls = cmls.assign_coords({'length': ('cml_id', lengths)})   
cmls = cmls.assign_coords({'site_0_lat': ('cml_id', cmls.site_a_latitude.values)})   
cmls = cmls.assign_coords({'site_0_lon': ('cml_id', cmls.site_a_longitude.values)})   
cmls = cmls.assign_coords({'site_1_lat': ('cml_id', cmls.site_b_latitude.values)})   
cmls = cmls.assign_coords({'site_1_lon': ('cml_id', cmls.site_b_longitude.values)})  
cmls = cmls.assign_coords({'frequency': ('cml_id', cmls.frequency.values)})  
cmls = cmls.assign_coords({'polarisation': ('cml_id', cmls.polarization.values)})  

In [11]:
cmls = cmls.drop_vars(['site_b_latitude', 'site_b_longitude', 'site_a_latitude', 'site_a_longitude', 'polarization'])

### Sort metadata by name and channel

In [12]:
# function for reading files
def read_file(file): 
    with libarchive.Archive(dir1 + file) as a:
        list = []
        for entry in a: # move to first file
            if entry.pathname[:7] == 'mw_data':
                data = BytesIO(a.read(size = entry.size))
                df = pd.read_csv(data, sep=",",index_col=False)[['Timestamp', 'Link', 'TX', 'RX']]
                df = df.rename(columns = {'Link': 'cml_id', 'TX': 'ts', 'RX': 'rs'})
                df['time'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d%H%M%S')
                list.append(df.drop(columns = ['Timestamp']))

        if len(list) == 0:
            return None
        else:
            return pd.concat(list, ignore_index=True)
    return None

In [13]:
# create the csv file with header and columnnames
file_counter = 0
files_list = []
ds_cml = None # dataset to store cml data in

# create xarray object to store data in
time_series = pd.date_range(start=start, end=end, freq='1T')
sublink_id = np.array(sorted(list(station_names))) 
cmls_pairs = []
for link_1 in sublink_id:
    cmls_pairs.append([0, 0]) # dummy list for cleaner code
    cmls_pairs[-1][0] = link_1
    link1_name_site_a = link_1.split('.')[0]
    link1_name_site_b = link_1.split('.')[1]

    for link_2 in sublink_id: # go through the whole list from beginning
        link2_name_site_a = link_2.split('.')[0]
        link2_name_site_b = link_2.split('.')[1]
        sita_a_is_site_b = link2_name_site_a == link1_name_site_b
        sita_b_is_site_a = link1_name_site_a == link2_name_site_b
        if sita_a_is_site_b and sita_b_is_site_a:
            cmls_pairs[-1][1] = link_2


# then remove duplicates
drop_indx_pair = []

# iterate the next n links
for i, sublink1 in enumerate(cmls_pairs):
    sublink1_a, sublink1_b = sublink1[0], sublink1[1]
    
    # search if sublink has a corresponding pair
    sublink_has_pair = False
    c = 0
    while (c < len(cmls_pairs)) and (not sublink_has_pair):
        sublink_check_a, sublink_check_b = cmls_pairs[c]
        
        # a fellow pair is alwas swiched
        if (sublink1_a == sublink_check_b) and (sublink1_b == sublink_check_a):
            sublink_has_pair = True
        c +=1
    c -= 1
    
    # if the sublink has pair
    if sublink_has_pair:
        if sorted([i, c]) not in drop_indx_pair:
            drop_indx_pair.append(sorted([i, c]))
    

drop_indx = [drop_indx_pair[i][0] for i in range(len(drop_indx_pair))]
sublink_id_pairs = np.array([cmls_pairs[i] for i in range(len(cmls_pairs)) if i not in drop_indx])

In [14]:
# Note: We can get channel 2 name by just swiching the two stations

In [15]:
# dataset for the two channels
ds_cml2chl = xr.Dataset(
    data_vars= dict(
        rs=(["cml_id", 'sublink_id', 'time'], np.zeros([sublink_id_pairs.shape[0], 2, time_series.size]).astype('float32')*np.nan),
        ts=(["cml_id", 'sublink_id', 'time'], np.zeros([sublink_id_pairs.shape[0], 2, time_series.size]).astype('float32')*np.nan), 
    ),
    coords=dict(
        cml_id = sublink_id_pairs[:, 0],
        sublink_id = ['chl_1', 'chl_2'],
        time = time_series,

        length = ('cml_id', np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan),
        site_0_lat = ('cml_id', np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan),
        site_0_lon = ('cml_id', np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan),
        site_1_lat = ('cml_id', np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan),
        site_1_lon = ('cml_id', np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan),
        frequency = ('cml_id', np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan),
        polarisation = ('cml_id', (np.zeros(sublink_id_pairs[:, 0].shape[0])*np.nan).astype(str)),
        
    ),
)

# store the complete timeseries in this dataset (faster reading from files)
ds_cml = xr.Dataset(
    data_vars= dict(
        rs=(["cml_id", 'time'], np.zeros([sublink_id.size, time_series.size]).astype('float32')*np.nan),
        ts=(["cml_id", 'time'], np.zeros([sublink_id.size, time_series.size]).astype('float32')*np.nan), 
    ),
    coords=dict(
        cml_id = sublink_id,
        time = time_series,
    ),
)

In [16]:
# populate xarray object with metadata where metadata exists
cml_with_metadata = []

for cml_id in ds_cml2chl.cml_id.values:
    pop = False # wether to populate the dataset
    try:
        cml_metadata = cmls.sel(cml_id = cml_id)
        pop = True

    except:
        # swich names, could work in theory but does not seem to be the case in the june set atleast! 
        cml_id2 = cml_id.split('.')
        try:
            cml_metadata = cmls.sel(cml_id = join([cml_id2[1],  cml_id2[0]]))
            pop = True
        except:
            continue # nothing happened
    
    if pop:
        cml_with_metadata.append(cml_id)
        ds_cml2chl['length'].loc[{'cml_id': cml_id}] = cml_metadata.length.values
        ds_cml2chl['site_0_lat'].loc[{'cml_id': cml_id}] = cml_metadata.site_0_lat.values
        ds_cml2chl['site_0_lon'].loc[{'cml_id': cml_id}] = cml_metadata.site_0_lon.values
        ds_cml2chl['site_1_lat'].loc[{'cml_id': cml_id}] = cml_metadata.site_1_lat.values
        ds_cml2chl['site_1_lon'].loc[{'cml_id': cml_id}] = cml_metadata.site_1_lon.values
        ds_cml2chl['frequency'].loc[{'cml_id': cml_id}] = cml_metadata.frequency.values
        ds_cml2chl['polarisation'].loc[{'cml_id': cml_id}] = cml_metadata.polarisation.values   

In [17]:
# drop links without metadata (comment to keep the links)
# ds_cml2chl = ds_cml2chl.sel(cml_id = cml_with_metadata)    

### Read timeseries from file

In [18]:
file_counter = 0 # reset counter
for file in tqdm.tqdm(files):
    file_out = read_file(file)
    if file_out is not None:
        files_list.append( file_out )
        file_counter += 1

    if file_counter > 30: # read data in chunks
        df = pd.concat(files_list)
        files_list = [] # reset file container
        file_conter = 0 # reset counter
        
        df.set_index(['time', 'cml_id'], inplace=True)
        df = df.groupby(level=df.index.names).median() # when overlap use median

        ds = df.to_xarray() # use the nearest value to the full minute
        ds_rs = ds.rs.resample(time = '1T', skipna=True).nearest(tolerance = '0.5T').rename('rs')
        ds_ts = ds.ts.resample(time = '1T', skipna=True).nearest(tolerance = '0.5T').rename('ts')
        times_in_ds = ds_rs.time
        links_in_ds = ds_rs.cml_id

        # remove CMLs from timeseries chunk that is not present in allocated container ds_cml
        in_list = []
        for i in range(len(links_in_ds.values)):
            if links_in_ds.values[i] in ds_cml.cml_id.values:
                in_list.append(i)
        links_in_ds = links_in_ds[in_list]
        ds_rs = ds_rs.isel(cml_id = in_list)
        ds_ts = ds_ts.isel(cml_id = in_list)        
                
        # Existing data from the data chunk
        existing_data = ds_cml['rs'].loc[dict(time=times_in_ds, cml_id=links_in_ds)] 

        # where existing mean is nan we use the new data
        # if existing and new has data, use existing, for further advancements use the closest, but that would require that we store the time
        ds_cml['rs'].loc[dict(time=times_in_ds, cml_id=links_in_ds)] = xr.where(np.isnan(existing_data), ds_rs, existing_data)

        # repeat for tx
        existing_data = ds_cml['ts'].loc[dict(time=times_in_ds, cml_id=links_in_ds)] 
        ds_cml['ts'].loc[dict(time=times_in_ds, cml_id=links_in_ds)] = xr.where(np.isnan(existing_data), ds_ts, existing_data)
        
        # Then reset container and counter
        files_list = [] 
        file_counter = 0 


100%|███████████████████████████████████████████████████████████████████████████████████████████| 6712/6712 [48:47<00:00,  2.29it/s]


In [19]:
# fill data from ds_cml into ds_cml2chl
# strategy: for throught ds_cml2chl, name is chnl1, swich is channel 2:) 
for cml_id in tqdm.tqdm(ds_cml2chl.cml_id.values):

    cml_id2 = cml_id.split('.')
    try:
        cml_data_ch1 = ds_cml.sel(cml_id = cml_id)
        pop_ch1 = True
        
    except:
        pop_ch1 = False
        print('skip ', cml_id, ' .. something is wery wrong')
        
    try: # swiching possitions
        cml_id_chl2 = '.'.join([cml_id2[1],  cml_id2[0]])
        cml_data_ch2 = ds_cml.sel(cml_id = cml_id_chl2)
        pop_ch2 = True
    except:
        # missing channel 2
        pop_ch2 = False
    
    if pop_ch1 and pop_ch2: # both channels are present
        ds_cml2chl['rs'].loc[{'cml_id': cml_id, 'sublink_id':'chl_1'}] = cml_data_ch1.rs.values
        ds_cml2chl['ts'].loc[{'cml_id': cml_id, 'sublink_id':'chl_1'}] = cml_data_ch1.ts.values
        ds_cml2chl['rs'].loc[{'cml_id': cml_id, 'sublink_id':'chl_2'}] = cml_data_ch2.rs.values
        ds_cml2chl['ts'].loc[{'cml_id': cml_id, 'sublink_id':'chl_2'}] = cml_data_ch2.ts.values

    # by construction channel 1 always has data
    elif pop_ch1:
        ds_cml2chl['rs'].loc[{'cml_id': cml_id, 'sublink_id':'chl_1'}] = cml_data_ch1.rs.values
        ds_cml2chl['ts'].loc[{'cml_id': cml_id, 'sublink_id':'chl_1'}] = cml_data_ch1.ts.values


100%|██████████████████████████████████████████████████████████████████████████████████████████| 2777/2777 [00:04<00:00, 597.64it/s]


In [20]:
# Store it 
if period == 'winter':
    ds_cml2chl.to_netcdf('/home/erlend/offline_data/cml_telia/cml_norway_2021-12.nc')
if period == 'summer':
    ds_cml2chl.to_netcdf('/home/erlend/offline_data/cml_telia/cml_norway_2022-06.nc')


# Eksport WKT

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import xarray as xr
import pandas as pd
import datetime
import tqdm
import pickle
import libarchive

ModuleNotFoundError: No module named 'libarchive'

In [12]:
df = pd.read_csv('/home/erlend/offline_data/cml_telia/meta_2022_June.csv')

In [14]:
linestring = ['LINESTRING(' + str(df['far_longitude'][i]) + ' ' + str(df['far_latitude'][i]) + 
              ',' + str(df['near_longitude'][i]) + ' ' + str(df['near_latitude'][i]) + ')' for i in range(df.link_id.size)]

In [15]:
df_out = pd.DataFrame(linestring)
df_out['cml_id'] = df.link_id # add names, can also add more info

In [16]:
df_out.to_csv('/home/erlend/offline_data/cml_telia/meta_2022_qgis', sep=';')

In [48]:
ds_cml2chl = xr.open_dataset('/home/erlend/offline_data/cml_telia/cml_norway_2021-12.nc').load()
# drop if nan in coordinates ( then we dont know possition)
cml_nan = np.isnan(ds_cml2chl.site_0_lat).values
cml_keep = ds_cml2chl.cml_id.values[~cml_nan]
ds_cml2chl = ds_cml2chl.sel(cml_id = cml_keep)

In [54]:
linestring = ['LINESTRING(' + str(ds_cml2chl['site_1_lon'][i].values) + ' ' + str(ds_cml2chl['site_1_lat'][i].values) + 
              ',' + str(ds_cml2chl['site_0_lon'][i].values) + ' ' + str(ds_cml2chl['site_0_lat'][i].values) + ')' for i in range(ds_cml2chl.cml_id.size)]



In [60]:
df_out = pd.DataFrame(linestring)
df_out['cml_id'] = cml_keep # add names, can also add more info

In [62]:
df_out.to_csv('/home/erlend/offline_data/cml_telia/meta_2022_June_qgis', sep=';')

In [17]:
ds_cml2chl = xr.open_dataset('/home/erlend/offline_data/cml_telia/cml_norway_2021-12.nc').load()


In [21]:
3np.isnan(ds_cml2chl.site_0_lat).sum()

<xarray.DataArray 'site_0_lat' ()>
array(154)